## Signal Autoencoder Training for Cross-check step

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
import torch.utils.data as utils

In [3]:
f = pd.read_hdf("/data/t3home000/spark/LHCOlympics_previous/LHC-Olympics/Code/Nsubjettiness_mjj.h5")

In [4]:
f.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,98.677270,0.528903,0.788281,0.904471,4.241889,136.0,1285.895950,53.519023,0.668562,0.735745,0.755674,1.895988,128.0,1282.286017,2577.571899,0.0
1,584.595432,0.345626,0.463461,0.865982,1.069972,320.0,1334.493332,405.034096,0.264362,0.793461,0.830032,1.377217,348.0,1306.137883,3807.507389,0.0
2,159.597526,0.677692,0.690707,0.695322,1.310040,332.0,678.557182,113.768840,0.713481,0.922610,0.782783,1.887494,236.0,1072.462085,1710.965414,0.0
3,515.237299,0.091038,0.784454,0.860716,1.102743,248.0,1284.020224,161.648798,0.727507,0.719564,0.870109,1.997360,352.0,1217.031950,2603.379037,0.0
4,142.420213,0.507714,0.522686,0.904070,1.853319,220.0,1087.658980,105.721163,0.344534,0.614579,0.863765,1.113248,204.0,1205.343324,3294.162200,1.0


In [5]:
dt = f.values

In [6]:
dt[:,0] = (dt[:,0]-np.mean(dt[:,0]))/np.std(dt[:,0])
dt[:,1] = (dt[:,1]-np.mean(dt[:,1]))/np.std(dt[:,1])
dt[:,2] = (dt[:,2]-np.mean(dt[:,2]))/np.std(dt[:,2])
dt[:,3] = (dt[:,3]-np.mean(dt[:,3]))/np.std(dt[:,3])
dt[:,4] = (dt[:,4]-np.mean(dt[:,4]))/np.std(dt[:,4])
dt[:,5] = (dt[:,5]-np.mean(dt[:,5]))/np.std(dt[:,5])

dt[:,7] = (dt[:,7]-np.mean(dt[:,7]))/np.std(dt[:,7])
dt[:,8] = (dt[:,8]-np.mean(dt[:,8]))/np.std(dt[:,8])
dt[:,9] = (dt[:,9]-np.mean(dt[:,9]))/np.std(dt[:,9])
dt[:,10] = (dt[:,10]-np.mean(dt[:,10]))/np.std(dt[:,10])
dt[:,11] = (dt[:,11]-np.mean(dt[:,11]))/np.std(dt[:,11])
dt[:,12] = (dt[:,12]-np.mean(dt[:,12]))/np.std(dt[:,12])

In [7]:
idx = dt[:,15]
bkg_idx = np.where(idx==0)[0]
signal_idx = np.where(idx==1)[0]

In [8]:
from sklearn.model_selection import train_test_split

# Architecture Definitions

In [9]:
#second architecture
class Encoder_jet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(12, 48)
        self.linear2 = nn.Linear(48, 30)
        self.linear3 = nn.Linear(30, 20)
        self.linear4 = nn.Linear(20, 10)
        self.linear5 = nn.Linear(10, 6)
        self.linear6 = nn.Linear(6, 3)

    def forward(self, x):
        x = F.leaky_relu(self.linear1(x))
        x = F.leaky_relu(self.linear2(x))
        x = F.leaky_relu(self.linear3(x))
        x = F.leaky_relu(self.linear4(x))
        x = F.leaky_relu(self.linear5(x))
        x = F.tanh(self.linear6(x))
        return x

In [10]:
class Decoder_jet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3, 6)
        self.linear2 = nn.Linear(6, 10)
        self.linear3 = nn.Linear(10, 20)
        self.linear4 = nn.Linear(20, 30)
        self.linear5 = nn.Linear(30, 48)
        self.linear6 = nn.Linear(48, 12)
        

    def forward(self, x):
        x = F.leaky_relu(self.linear1(x))
        x = F.leaky_relu(self.linear2(x))
        x = F.leaky_relu(self.linear3(x))
        x = F.leaky_relu(self.linear4(x))
        x = F.leaky_relu(self.linear5(x))
        x = self.linear6(x)
        return x

In [11]:
class PlanarFlow(nn.Module):
    def __init__(self, D):
        super().__init__()
        self.D = D

    def forward(self, z, lamda):
        '''
        z - latents from prev layer
        lambda - Flow parameters (b, w, u)
        b - scalar
        w - vector
        u - vector
        '''
        b = lamda[:, :1]
        w, u = lamda[:, 1:].chunk(2, dim=1)

        # Forward
        # f(z) = z + u tanh(w^T z + b)
        transf = F.tanh(
            z.unsqueeze(1).bmm(w.unsqueeze(2))[:, 0] + b
        )
        f_z = z + u * transf

        # Inverse
        # psi_z = tanh' (w^T z + b) w
        psi_z = (1 - transf ** 2) * w
        log_abs_det_jacobian = torch.log(
            (1 + psi_z.unsqueeze(1).bmm(u.unsqueeze(2))).abs()
        )

        return f_z, log_abs_det_jacobian

In [12]:
class NormalizingFlow(nn.Module):
    def __init__(self, K, D):
        super().__init__()
        self.flows = nn.ModuleList([PlanarFlow(D) for i in range(K)])

    def forward(self, z_k, flow_params):
        # ladj -> log abs det jacobian
        sum_ladj = 0
        for i, flow in enumerate(self.flows):
            z_k, ladj_k = flow(z_k, flow_params[i])
            sum_ladj += ladj_k

        return z_k, sum_ladj

In [13]:
class VAE_NF(nn.Module):
    def __init__(self, K, D):
        super().__init__()
        self.dim = D
        self.K = K
        self.encoder = nn.Sequential(
            nn.Linear(22, 96),
            nn.LeakyReLU(True),
            nn.Linear(96, 48),
            nn.LeakyReLU(True),            
            nn.Linear(48, D * 2 + K * (D * 2 + 1))
        )

        self.decoder = nn.Sequential(
            nn.Linear(D, 48),
            nn.LeakyReLU(True),
            nn.Linear(48, 96),
            nn.LeakyReLU(True),
            nn.Linear(96, 22),
            nn.Sigmoid()
        )

        self.flows = NormalizingFlow(K, D)

    def forward(self, x):
        # Run Encoder and get NF params
        enc = self.encoder(x)
        mu = enc[:, :self.dim]
        log_var = enc[:, self.dim: self.dim * 2]
        flow_params = enc[:, 2 * self.dim:].chunk(self.K, dim=1)

        # Re-parametrize
        sigma = (log_var * .5).exp()
        z = mu + sigma * torch.randn_like(sigma)
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        # Construct more expressive posterior with NF
        z_k, sum_ladj = self.flows(z, flow_params)
        kl_div = kl_div / x.size(0) - sum_ladj.mean()  # mean over batch

        # Run Decoder
        x_prime = self.decoder(z_k)
        return x_prime, kl_div

In [1]:
def train_with_idx(dt, train_idx, bb_idx, num_sig):
    data_train_temp = torch.tensor(dt[train_idx,:])
    data_train_temp_1 = data_train_temp.t()[0:6].t()
    data_train_temp_2 = data_train_temp.t()[7:13].t()
    data_train = torch.cat((data_train_temp_1,data_train_temp_2),dim=1)
    bs = 100
    my_dataset = utils.TensorDataset(data_train,data_train) 
    my_dataloader = utils.DataLoader(my_dataset, batch_size=bs, shuffle=True)
    sig_model_enc = Encoder_jet().cuda()
    sig_model_dec = Decoder_jet().cuda()
    lrs = 1e-4
    sig_optimizer_enc = optim.Adam(sig_model_enc.parameters(), lr = lrs)
    sig_optimizer_dec = optim.Adam(sig_model_dec.parameters(), lr = lrs)
    sig_loss = nn.MSELoss()
    epochs = 80
    patience_counter = 0
    min_loss = 9999
    for epoch in range(epochs):
        sig_model_enc.train()
        sig_model_dec.train()
        for i, dtt in enumerate(my_dataloader):
            sig_optimizer_enc.zero_grad()
            sig_optimizer_dec.zero_grad()
            
            inp = dtt[0].float().cuda()
            output = dtt[1].float().cuda()
            ls = sig_loss(sig_model_dec(sig_model_enc(inp)),output)
            ls.backward()
            sig_optimizer_enc.step()
            sig_optimizer_dec.step()
        
        print("Epoch: " + str(epoch), "Loss Training: " + str(ls.data.cpu().numpy()))
        if ls.data.cpu().numpy() < min_loss:
            min_loss = ls.data.cpu().numpy()
            patience_counter = 0
            torch.save(sig_model_enc.state_dict(), f"sigae_enc_{num_sig}.h5")
            torch.save(sig_model_dec.state_dict(), f"sigae_dec_{num_sig}.h5")
            print('patience counter: ',patience_counter)
    
            
        else:
            patience_counter += 1
            print('patience counter: ',patience_counter)
            if patience_counter >= 10:
                print('maximum patience reached')
                break
            
    data_bb_temp = torch.tensor(dt[bb_idx,:])
    data_bb_temp_1 = data_bb_temp.t()[0:6].t()
    data_bb_temp_2 = data_bb_temp.t()[7:13].t()
    data_bb = torch.cat((data_bb_temp_1,data_bb_temp_2),dim=1)
    signalAE_loss = torch.mean((sig_model_dec(sig_model_enc(data_bb.float().cuda()))-data_bb.float().cuda())**2,dim=1).data.cpu().numpy()
    np.save(f'sigaeloss_numsig_{num_sig}_bb',signalAE_loss)
    
    signalAE_trainloss = torch.mean((sig_model_dec(sig_model_enc(data_train.float().cuda()))-data_train.float().cuda())**2,dim=1).data.cpu().numpy()
    np.save(f'sigaeloss_numsig_{num_sig}_trainloss',signalAE_trainloss)
    
    f = pd.read_hdf("/data/t3home000/spark/LHCOlympics/data/MassRatio_pureBkg.h5")
    dt = f.values
    dt[:,1] = (dt[:,1]-np.mean(dt[:,1]))/np.std(dt[:,1])
    dt[:,2] = (dt[:,2]-np.mean(dt[:,2]))/np.std(dt[:,2])
    dt[:,3] = (dt[:,3]-np.mean(dt[:,3]))/np.std(dt[:,3])
    dt[:,4] = (dt[:,4]-np.mean(dt[:,4]))/np.std(dt[:,4])
    dt[:,5] = (dt[:,5]-np.mean(dt[:,5]))/np.std(dt[:,5])
    dt[:,6] = (dt[:,6]-np.mean(dt[:,6]))/np.std(dt[:,6])

    dt[:,14] = (dt[:,14]-np.mean(dt[:,14]))/np.std(dt[:,14])
    dt[:,15] = (dt[:,15]-np.mean(dt[:,15]))/np.std(dt[:,15])
    dt[:,16] = (dt[:,16]-np.mean(dt[:,16]))/np.std(dt[:,16])
    dt[:,17] = (dt[:,17]-np.mean(dt[:,17]))/np.std(dt[:,17])
    dt[:,18] = (dt[:,18]-np.mean(dt[:,18]))/np.std(dt[:,18])
    dt[:,19] = (dt[:,19]-np.mean(dt[:,19]))/np.std(dt[:,19])
    data = torch.tensor(dt)
    data_train_x_1 = data.t()[1:7].t()
    data_train_x_2 = data.t()[14:20].t()
    data_bkg = torch.cat((data_train_x_1,data_train_x_2),dim=1)
    signalAE_loss_purebkg = torch.mean((sig_model_dec(sig_model_enc(data_bkg.float().cuda()))-data_bkg.float().cuda())**2,dim=1).data.cpu().numpy()
    np.save(f'sigaeloss_numsig_{num_sig}_bkg',signalAE_loss_purebkg)
    

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.get_device_name(0)

cuda:0


'GeForce RTX 2080 Ti'

In [16]:
BATCH_SIZE = 1000
N_EPOCHS = 80
PRINT_INTERVAL = 2000
NUM_WORKERS = 4
LR = 1e-4
MODEL = 'VAE-NF'  # VAE-NF | VAE

N_FLOWS = 2
Z_DIM = 3

n_steps = 0
model = VAE_NF(N_FLOWS, Z_DIM).cuda()

In [17]:
model.load_state_dict(torch.load("/data/t3home000/spark/LHCOlympics_previous/LHC-Olympics/Code/models/VAE_nf_v1.h5"))

<All keys matched successfully>

In [25]:
def get_mass_and_loss_bkg(inputstring):
    f_in = pd.read_hdf(inputstring)
    dt_in = f_in.values
    dt_in[:,1] = (dt_in[:,1]-np.mean(dt_in[:,1]))/np.std(dt_in[:,1])
    dt_in[:,2] = (dt_in[:,2]-np.mean(dt_in[:,2]))/np.std(dt_in[:,2])
    dt_in[:,3] = (dt_in[:,3]-np.mean(dt_in[:,3]))/np.std(dt_in[:,3])
    dt_in[:,4] = (dt_in[:,4]-np.mean(dt_in[:,4]))/np.std(dt_in[:,4])
    dt_in[:,5] = (dt_in[:,5]-np.mean(dt_in[:,5]))/np.std(dt_in[:,5])
    dt_in[:,6] = (dt_in[:,6]-np.mean(dt_in[:,6]))/np.std(dt_in[:,6])

    dt_in[:,8] = (dt_in[:,8]-np.mean(dt_in[:,8]))/np.std(dt_in[:,8])
    dt_in[:,9] = (dt_in[:,9]-np.mean(dt_in[:,9]))/np.std(dt_in[:,9])
    dt_in[:,10] = (dt_in[:,10]-np.mean(dt_in[:,10]))/np.std(dt_in[:,10])
    dt_in[:,11] = (dt_in[:,11]-np.mean(dt_in[:,11]))/np.std(dt_in[:,11])
    dt_in[:,12] = (dt_in[:,12]-np.mean(dt_in[:,12]))/np.std(dt_in[:,12])

    dt_in[:,14] = (dt_in[:,14]-np.mean(dt_in[:,14]))/np.std(dt_in[:,14])
    dt_in[:,15] = (dt_in[:,15]-np.mean(dt_in[:,15]))/np.std(dt_in[:,15])
    dt_in[:,16] = (dt_in[:,16]-np.mean(dt_in[:,16]))/np.std(dt_in[:,16])
    dt_in[:,17] = (dt_in[:,17]-np.mean(dt_in[:,17]))/np.std(dt_in[:,17])
    dt_in[:,18] = (dt_in[:,18]-np.mean(dt_in[:,18]))/np.std(dt_in[:,18])
    dt_in[:,19] = (dt_in[:,19]-np.mean(dt_in[:,19]))/np.std(dt_in[:,19])
    
    dt_in[:,21] = (dt_in[:,21]-np.mean(dt_in[:,21]))/np.std(dt_in[:,21])
    dt_in[:,22] = (dt_in[:,22]-np.mean(dt_in[:,22]))/np.std(dt_in[:,22])
    dt_in[:,23] = (dt_in[:,23]-np.mean(dt_in[:,23]))/np.std(dt_in[:,23])
    dt_in[:,24] = (dt_in[:,24]-np.mean(dt_in[:,24]))/np.std(dt_in[:,24])
    dt_in[:,25] = (dt_in[:,25]-np.mean(dt_in[:,25]))/np.std(dt_in[:,25])


    
    
    total_in = torch.tensor(dt_in)
    total_in_train_x_1 = total_in.t()[1:7].t()
    total_in_train_x_2 = total_in.t()[8:13].t()
    total_in_train_x_3 = total_in.t()[14:20].t()
    total_in_train_x_4 = total_in.t()[21:26].t()
    total_in_selection = torch.cat((total_in_train_x_1,total_in_train_x_2,total_in_train_x_3,total_in_train_x_4),dim=1)
    
    loss_total_in = torch.mean((model(total_in_selection.float().cuda())[0]-
                       total_in_selection.float().cuda())**2,dim=1).data.cpu().numpy()
    
    f_in = pd.read_hdf(inputstring)
    dt_in = f_in.values
    
    return dt_in[:,0], dt_in[:,10], dt_in[:,23], dt_in[:,9], dt_in[:,22], loss_total_in

In [58]:
def get_mass_and_loss_bb(inputstring,bb_idx):
    f_in = pd.read_hdf(inputstring)
    dt_in = f_in.values
    print('shape',bb_idx.shape)
    print('before: ', dt_in.shape)
    dt_in = dt_in[bb_idx,:]
    print('after: ',dt_in.shape)
    dt_in[:,1] = (dt_in[:,1]-np.mean(dt_in[:,1]))/np.std(dt_in[:,1])
    dt_in[:,2] = (dt_in[:,2]-np.mean(dt_in[:,2]))/np.std(dt_in[:,2])
    dt_in[:,3] = (dt_in[:,3]-np.mean(dt_in[:,3]))/np.std(dt_in[:,3])
    dt_in[:,4] = (dt_in[:,4]-np.mean(dt_in[:,4]))/np.std(dt_in[:,4])
    dt_in[:,5] = (dt_in[:,5]-np.mean(dt_in[:,5]))/np.std(dt_in[:,5])
    dt_in[:,6] = (dt_in[:,6]-np.mean(dt_in[:,6]))/np.std(dt_in[:,6])

    dt_in[:,8] = (dt_in[:,8]-np.mean(dt_in[:,8]))/np.std(dt_in[:,8])
    dt_in[:,9] = (dt_in[:,9]-np.mean(dt_in[:,9]))/np.std(dt_in[:,9])
    dt_in[:,10] = (dt_in[:,10]-np.mean(dt_in[:,10]))/np.std(dt_in[:,10])
    dt_in[:,11] = (dt_in[:,11]-np.mean(dt_in[:,11]))/np.std(dt_in[:,11])
    dt_in[:,12] = (dt_in[:,12]-np.mean(dt_in[:,12]))/np.std(dt_in[:,12])

    dt_in[:,14] = (dt_in[:,14]-np.mean(dt_in[:,14]))/np.std(dt_in[:,14])
    dt_in[:,15] = (dt_in[:,15]-np.mean(dt_in[:,15]))/np.std(dt_in[:,15])
    dt_in[:,16] = (dt_in[:,16]-np.mean(dt_in[:,16]))/np.std(dt_in[:,16])
    dt_in[:,17] = (dt_in[:,17]-np.mean(dt_in[:,17]))/np.std(dt_in[:,17])
    dt_in[:,18] = (dt_in[:,18]-np.mean(dt_in[:,18]))/np.std(dt_in[:,18])
    dt_in[:,19] = (dt_in[:,19]-np.mean(dt_in[:,19]))/np.std(dt_in[:,19])
    
    dt_in[:,21] = (dt_in[:,21]-np.mean(dt_in[:,21]))/np.std(dt_in[:,21])
    dt_in[:,22] = (dt_in[:,22]-np.mean(dt_in[:,22]))/np.std(dt_in[:,22])
    dt_in[:,23] = (dt_in[:,23]-np.mean(dt_in[:,23]))/np.std(dt_in[:,23])
    dt_in[:,24] = (dt_in[:,24]-np.mean(dt_in[:,24]))/np.std(dt_in[:,24])
    dt_in[:,25] = (dt_in[:,25]-np.mean(dt_in[:,25]))/np.std(dt_in[:,25])


    
    
    total_in = torch.tensor(dt_in)
    total_in_train_x_1 = total_in.t()[1:7].t()
    total_in_train_x_2 = total_in.t()[8:13].t()
    total_in_train_x_3 = total_in.t()[14:20].t()
    total_in_train_x_4 = total_in.t()[21:26].t()
    total_in_selection = torch.cat((total_in_train_x_1,total_in_train_x_2,total_in_train_x_3,total_in_train_x_4),dim=1)
    
    loss_total_in = torch.mean((model(total_in_selection.float().cuda())[0]-
                       total_in_selection.float().cuda())**2,dim=1).data.cpu().numpy()
    
    return dt_in[:,0], loss_total_in

In [59]:
def blackboxmaker(dt, sig_idx, bkg_idx, num_sig):
    train, test = train_test_split(sig_idx,test_size=num_sig)
    print(train.shape)
    blackboxidx = np.append(bkg_idx,test)
    train_with_idx(dt, train, blackboxidx, num_sig)
    bbmass, bbloss = get_mass_and_loss_bb("/data/t3home000/spark/LHCOlympics/data/MassRatio_RandD.h5",blackboxidx)
    purebkgmass, purebkgmmdt1, purebkgmmdt2, purebkgprun1,purebkgprun2, purebkgloss = get_mass_and_loss_bkg("/data/t3home000/spark/LHCOlympics/data/MassRatio_pureBkg.h5")
    np.save(f'mass_numsig_{num_sig}',bbmass)
    np.save('mass_bkg.npy',purebkgmass)
    np.save(f'bkgaeloss_numsig_{num_sig}',bbloss)
    np.save('bkgaeloss_bkg.npy',purebkgloss)
    #np.save(f'blackboxidx_{num_sig}', blackboxidx )

In [60]:
blackboxmaker(dt, signal_idx,bkg_idx, 10000)

(90000,)
Epoch: 0 Loss Training: 0.7304543
patience counter:  0
Epoch: 1 Loss Training: 0.69541687
patience counter:  0
Epoch: 2 Loss Training: 0.6275796
patience counter:  0
Epoch: 3 Loss Training: 0.6517631
patience counter:  1
Epoch: 4 Loss Training: 0.6588153
patience counter:  2
Epoch: 5 Loss Training: 0.5683471
patience counter:  0
Epoch: 6 Loss Training: 0.58594525
patience counter:  1
Epoch: 7 Loss Training: 0.6989092
patience counter:  2
Epoch: 8 Loss Training: 0.5892665
patience counter:  3
Epoch: 9 Loss Training: 0.6124245
patience counter:  4
Epoch: 10 Loss Training: 0.5042583
patience counter:  0
Epoch: 11 Loss Training: 0.54556894
patience counter:  1
Epoch: 12 Loss Training: 0.5205042
patience counter:  2
Epoch: 13 Loss Training: 0.5738252
patience counter:  3
Epoch: 14 Loss Training: 0.4737109
patience counter:  0
Epoch: 15 Loss Training: 0.46958837
patience counter:  0
Epoch: 16 Loss Training: 0.5386655
patience counter:  1
Epoch: 17 Loss Training: 0.44508067
patience 

In [61]:
blackboxmaker(dt, signal_idx,bkg_idx, 1000)

(99000,)
Epoch: 0 Loss Training: 0.72924364
patience counter:  0
Epoch: 1 Loss Training: 0.7680662
patience counter:  1
Epoch: 2 Loss Training: 0.6995684
patience counter:  0
Epoch: 3 Loss Training: 0.6701377
patience counter:  0
Epoch: 4 Loss Training: 0.5060015
patience counter:  0
Epoch: 5 Loss Training: 0.65874213
patience counter:  1
Epoch: 6 Loss Training: 0.650142
patience counter:  2
Epoch: 7 Loss Training: 0.6857457
patience counter:  3
Epoch: 8 Loss Training: 0.6025545
patience counter:  4
Epoch: 9 Loss Training: 0.5686222
patience counter:  5
Epoch: 10 Loss Training: 0.46327937
patience counter:  0
Epoch: 11 Loss Training: 0.5209136
patience counter:  1
Epoch: 12 Loss Training: 0.51107574
patience counter:  2
Epoch: 13 Loss Training: 0.49241415
patience counter:  3
Epoch: 14 Loss Training: 0.526607
patience counter:  4
Epoch: 15 Loss Training: 0.42267498
patience counter:  0
Epoch: 16 Loss Training: 0.50684255
patience counter:  1
Epoch: 17 Loss Training: 0.40131733
patience

In [62]:
blackboxmaker(dt, signal_idx,bkg_idx, 100)

(99900,)
Epoch: 0 Loss Training: 0.84763885
patience counter:  0
Epoch: 1 Loss Training: 0.6701685
patience counter:  0
Epoch: 2 Loss Training: 0.6765547
patience counter:  1
Epoch: 3 Loss Training: 0.6278823
patience counter:  0
Epoch: 4 Loss Training: 0.6131298
patience counter:  0
Epoch: 5 Loss Training: 0.5800111
patience counter:  0
Epoch: 6 Loss Training: 0.6385804
patience counter:  1
Epoch: 7 Loss Training: 0.51943153
patience counter:  0
Epoch: 8 Loss Training: 0.4642059
patience counter:  0
Epoch: 9 Loss Training: 0.5073471
patience counter:  1
Epoch: 10 Loss Training: 0.5965235
patience counter:  2
Epoch: 11 Loss Training: 0.5109427
patience counter:  3
Epoch: 12 Loss Training: 0.42334425
patience counter:  0
Epoch: 13 Loss Training: 0.43196702
patience counter:  1
Epoch: 14 Loss Training: 0.45328492
patience counter:  2
Epoch: 15 Loss Training: 0.5000589
patience counter:  3
Epoch: 16 Loss Training: 0.51796055
patience counter:  4
Epoch: 17 Loss Training: 0.55099756
patienc

In [63]:
blackboxmaker(dt, signal_idx,bkg_idx, 10)

(99990,)
Epoch: 0 Loss Training: 0.7569997
patience counter:  0
Epoch: 1 Loss Training: 0.7088797
patience counter:  0
Epoch: 2 Loss Training: 0.67568237
patience counter:  0
Epoch: 3 Loss Training: 0.68582803
patience counter:  1
Epoch: 4 Loss Training: 0.5446771
patience counter:  0
Epoch: 5 Loss Training: 0.6457723
patience counter:  1
Epoch: 6 Loss Training: 0.5585306
patience counter:  2
Epoch: 7 Loss Training: 0.6205943
patience counter:  3
Epoch: 8 Loss Training: 0.42225286
patience counter:  0
Epoch: 9 Loss Training: 0.526603
patience counter:  1
Epoch: 10 Loss Training: 0.41971257
patience counter:  0
Epoch: 11 Loss Training: 0.47061148
patience counter:  1
Epoch: 12 Loss Training: 0.3851867
patience counter:  0
Epoch: 13 Loss Training: 0.5195433
patience counter:  1
Epoch: 14 Loss Training: 0.50439763
patience counter:  2
Epoch: 15 Loss Training: 0.4914897
patience counter:  3
Epoch: 16 Loss Training: 0.43288893
patience counter:  4
Epoch: 17 Loss Training: 0.46080163
patienc